<a href="https://colab.research.google.com/github/aswinkler/k-net/blob/main/notebooks/module_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook tests the functionality of the modules used to build a k-net

In [4]:
%run ../tests/test_dual_number

All DualNumber tests passed.


In [5]:
%run ../tests/test_dual_tensor

All DualTensor tests passed.


In [6]:
%run ../tests/test_dual_functions

All DualFunction tests passed.


In [1]:
%run ../tests//test_bspline

B-spline tests passed.


In [3]:
%run ../tests/test_ka_inner_layer

Real input test passed.
Dual input test passed.


In [6]:
%run ../tests/test_ka_full_layer

KAFullLayer real input test passed.
KAFullLayer dual input test passed.


In [11]:
%run ../tests/test_kan_model

Output real: [[-3.97997929]
 [ 0.        ]]
Output dual: [[0.]
 [0.]]
